# Optilab tutorial: creating your own optimizer
This tutorial aims to explain more custom usecases for optilab. In this notebook you will learn how to:
- create your own objective function,
- create your own optimizer,
- perform optimization using optilab API and your custom components.

## Creating a custom objective function
Optilab allows you to create your own custom objective function by creating a class inheriting the `ObjectiveFunction` base class. Methods `__init__` and `__call__` need to be overloaded. Let's create an example objective function:

In [ ]:
# import dependencies
import numpy as np
from optilab.data_classes import Point
from optilab.functions import ObjectiveFunction

# custom objective function
class SimpleObjectiveFunction(ObjectiveFunction):
    def __init__(self, dim: int):
        # setting necessary metadata for the function
        super().__init__('custom_function', dim)

    def __call__(self, point: Point) -> Point:
        # incrementing call counter
        super().__call__(point)

        # actual function value calculation
        return Point(
            x = point.x,
            y = sum([x_i ** i for i, x_i in enumerate(point.x)]),
            is_evaluated = True
        )        

By overloading the constructor of the function, name and dimensionality TODO

In [ ]:
func = SimpleObjectiveFunction(10)
func.get_metadata()

TODO evaluate a point using the function

In [ ]:
my_point = Point([2.3] * 10)
print(func(my_point))

Let's now create a more complex function that may have some additional hyperparameters:

In [ ]:
# import for typehint
from optilab.data_classes import FunctionMetadata


class ComplexObjectiveFunction(ObjectiveFunction):
    def __init__(self, dim: int, exponent: float):
        # setting necessary metadata for the function
        super().__init__('custom_function', dim)
        self.exponent = exponent

    def get_metadata(self) -> FunctionMetadata:
        metadata = super().get_metadata()
        metadata.hyperparameters['exponent'] = self.exponent
        return metadata

    def __call__(self, point: Point) -> Point:
        # incrementing call counter
        super().__call__(point)

        # actual function value calculation
        return Point(
            x = point.x,
            y = sum([x_i ** self.exponent for x_i in point.x]),
            is_evaluated = True
        )  

Let's now evaluate out point

In [ ]:
comp = ComplexObjectiveFunction(10, 3)
comp(my_point)

See metadata

In [ ]:
comp.get_metadata()

## Creating a custom optimizer
TODO overload init and optimize, run_optimization is already done

In [ ]:
from optilab.optimizers import Optimizer

class CustomOptimizer:
    def __init__(self):
        pass

    def optimize(self):
        pass